*This notebook assumes to be launched inside the source root and it uses relative path to obtain other resources.*

Test MapD->PyGDF->matrix

In [1]:
PWD = !pwd

In [2]:
import sys
import os.path
from pprint import pprint

Add import path to MapD Thrift binding and Arrow schema

In [3]:
thirdparty_path = os.path.join(PWD[0], '..', 'thirdparty')
sys.path.append(thirdparty_path)
thirdparty_path

'/home/appuser/pygdf/notebooks/../thirdparty'

If `pygdf` cannot be imported, uncomment code below:

In [4]:
# pygdf_path = os.path.join(PWD[0], '..')
# sys.path.append(pygdf_path)

In [5]:
import pygdf

In [6]:
from thrift.protocol import TBinaryProtocol
from thrift.protocol import TJSONProtocol
from thrift.transport import TSocket
from thrift.transport import THttpClient
from thrift.transport import TTransport

In [7]:
from mapd import MapD
from mapd import ttypes

MapD connection

In [8]:
def get_client(host_or_uri, port, http):
  if http:
    transport = THttpClient.THttpClient(host_or_uri)
    protocol = TJSONProtocol.TJSONProtocol(transport)
  else:
    socket = TSocket.TSocket(host_or_uri, port)
    transport = TTransport.TBufferedTransport(socket)
    protocol = TBinaryProtocol.TBinaryProtocol(transport)

  client = MapD.Client(protocol)
  transport.open()
  return client

In [9]:
db_name = 'mapd'
user_name = 'mapd'
passwd = 'HyperInteractive'
hostname = 'localhost'
portno = 9091

client = get_client(hostname, portno, False)
session = client.connect(user_name, passwd, db_name)
print('Connection complete')

Connection complete


The Query

In [10]:
columns = """
INCEARN,RECTYPE,IPUMS_YEAR,DATANUM,SERIAL,NUMPREC,SUBSAMP,HHWT,HHTYPE,REPWT,ADJUST,CPI99,REGION,STATEICP,STATEFIP,COUNTY,COUNTYFIPS,METRO,METAREA,METAREAD,MET2013,MET2013ERR,CITY,CITYERR,CITYPOP,PUMA,PUMARES2MIG,STRATA,PUMASUPR,CONSPUMA,CPUMA0010,APPAL,APPALD,HOMELAND,MET2003,CNTRY,GQ,GQTYPE,GQTYPED,FARM,OWNERSHP,OWNERSHPD,MORTGAGE,MORTGAG2,COMMUSE,FARMPROD,ACREHOUS,MORTAMT1,MORTAMT2,TAXINCL,INSINCL,PROPINSR,PROPTX99,OWNCOST,RENT,RENTGRS,RENTMEAL,CONDOFEE,MOBLHOME,MOBLHOM2,MOBLOAN,SECRES,SECRESMO,SECRESRE,COSTELEC,COSTGAS,COSTWATR,COSTFUEL,PUBHOUS,RENTSUB,HEATSUB,LUNCHSUB,FOODSTMP,FDSTPAMT,VALUEH,LINGISOL,VACANCY,KITCHEN,KITCHENORIG,FRIDGE,FRIDGEORIG,SINK,STOVE,ROOMS,ROOMSORIG,PLUMBING,HOTWATER,SHOWER,TOILET,BUILTYR,BUILTYR2,UNITSSTR,BEDROOMS,BEDROOMSORIG,PHONE,PHONEORIG,CILAPTOP,CIHAND,CIOTHCOMP,CINETHH,CIMODEM,CISAT,CIDSL,CIFIBER,CIBRDBND,CIDIAL,CIOTHSVC,FUELHEAT,VEHICLES,SSMC,NFAMS,NSUBFAM,NCOUPLES,NMOTHERS,NFATHERS,MULTGEN,MULTGEND,CBNSUBFAM,REPWT1,REPWT2,REPWT3,REPWT4,REPWT5,REPWT6,REPWT7,REPWT8,REPWT9,REPWT10,REPWT11,REPWT12,REPWT13,REPWT14,REPWT15,REPWT16,REPWT17,REPWT18,REPWT19,REPWT20,REPWT21,REPWT22,REPWT23,REPWT24,REPWT25,REPWT26,REPWT27,REPWT28,REPWT29,REPWT30,REPWT31,REPWT32,REPWT33,REPWT34,REPWT35,REPWT36,REPWT37,REPWT38,REPWT39,REPWT40,REPWT41,REPWT42,REPWT43,REPWT44,REPWT45,REPWT46,REPWT47,REPWT48,REPWT49,REPWT50,REPWT51,REPWT52,REPWT53,REPWT54,REPWT55,REPWT56,REPWT57,REPWT58,REPWT59,REPWT60,REPWT61,REPWT62,REPWT63,REPWT64,REPWT65,REPWT66,REPWT67,REPWT68,REPWT69,REPWT70,REPWT71,REPWT72,REPWT73,REPWT74,REPWT75,REPWT76,REPWT77,REPWT78,REPWT79,REPWT80,RESPMODE,PERNUM,PERWT,SLWT,REPWTP,FAMSIZE,NCHILD,NCHLT5,FAMUNIT,ELDCH,YNGCH,NSIBS,MOMLOC,STEPMOM,MOMRULE,POPLOC,STEPPOP,POPRULE,SPLOC,SPRULE,SUBFAM,SFTYPE,SFRELATE,CBSUBFAM,CBSFTYPE,CBSFRELATE,RELATE,RELATED,SEX,AGE,AGEORIG,BIRTHQTR,MARST,BIRTHYR,MARRNO,MARRINYR,YRMARR,DIVINYR,WIDINYR,FERTYR,RACE,RACED,HISPAN,HISPAND,BPL,BPLD,ANCESTR1,ANCESTR1D,ANCESTR2,ANCESTR2D,CITIZEN,YRNATUR,YRIMMIG,YRSUSA1,YRSUSA2,SPOKEN_LANGUAGE,LANGUAGED,SPEAKENG,TRIBE,TRIBED,RACESING,RACESINGD,RACAMIND,RACASIAN,RACBLK,RACPACIS,RACWHT,RACOTHER,RACNUM,SCHOOL,EDUC,EDUCD,GRADEATT,GRADEATTD,SCHLTYPE,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,LABFORCE,OCC,OCC1950,OCC1990,OCC2010,IND,IND1950,IND1990,CLASSWKR,CLASSWKRD,OCCSOC,INDNAICS,WKSWORK1,WKSWORK2,UHRSWORK,WRKLSTWK,ABSENT,LOOKING,AVAILBLE,WRKRECAL,WORKEDYR,POVERTY,OCCSCORE,SEI,HWSEI,PRESGL,PRENT,ERSCOR50,ERSCOR90,EDSCOR50,EDSCOR90,NPBOSS50,NPBOSS90,MIGRATE1,MIGRATE1D,MIGPLAC1,MIGMET1,MIGTYPE1,MIGCITY1,MIGPUMS1,MIGPUMA1,MOVEDIN,MOVEDINORIG,DISABWRK,VETDISAB,DIFFREM,DIFFPHYS,DIFFMOB,DIFFCARE,DIFFSENS,DIFFEYE,DIFFHEAR,VETSTAT,VETSTATD,VET01LTR,VET95X00,VET90X01,VET90X95,VET75X90,VET80X90,VET75X80,VETVIETN,VET55X64,VETKOREA,VET47X50,VETWWII,VETOTHER,VETYRS,PWSTATE2,PWMETRO,PWCITY,PWTYPE,PWPUMA00,PWPUMAS,TRANWORK,CARPOOL,RIDERS,TRANTIME,DEPARTS,ARRIVES,GCHOUSE,GCMONTHS,GCRESPON,PROBAI,PROBAPI,PROBBLK,PROBOTH,PROBWHT,REPWTP1,REPWTP2,REPWTP3,REPWTP4,REPWTP5,REPWTP6,REPWTP7,REPWTP8,REPWTP9,REPWTP10,REPWTP11,REPWTP12,REPWTP13,REPWTP14,REPWTP15,REPWTP16,REPWTP17,REPWTP18,REPWTP19,REPWTP20,REPWTP21,REPWTP22,REPWTP23,REPWTP24,REPWTP25,REPWTP26,REPWTP27,REPWTP28,REPWTP29,REPWTP30,REPWTP31,REPWTP32,REPWTP33,REPWTP34,REPWTP35,REPWTP36,REPWTP37,REPWTP38,REPWTP39,REPWTP40,REPWTP41,REPWTP42,REPWTP43,REPWTP44,REPWTP45,REPWTP46,REPWTP47,REPWTP48,REPWTP49,REPWTP50,REPWTP51,REPWTP52,REPWTP53,REPWTP54,REPWTP55,REPWTP56,REPWTP57,REPWTP58,REPWTP59,REPWTP60,REPWTP61,REPWTP62,REPWTP63,REPWTP64,REPWTP65,REPWTP66,REPWTP67,REPWTP68,REPWTP69,REPWTP70,REPWTP71,REPWTP72,REPWTP73,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80
""".strip()
print(len(columns.split(',')))

447


In [11]:
query = "SELECT {} FROM ipums_easy WHERE INCEARN > 100;".format(columns)
print('Query is : ' + query)

# always use True for is columnar
results = client.sql_execute_cudf(session, query, device_id=0, first_n=-1)

Query is : SELECT INCEARN,RECTYPE,IPUMS_YEAR,DATANUM,SERIAL,NUMPREC,SUBSAMP,HHWT,HHTYPE,REPWT,ADJUST,CPI99,REGION,STATEICP,STATEFIP,COUNTY,COUNTYFIPS,METRO,METAREA,METAREAD,MET2013,MET2013ERR,CITY,CITYERR,CITYPOP,PUMA,PUMARES2MIG,STRATA,PUMASUPR,CONSPUMA,CPUMA0010,APPAL,APPALD,HOMELAND,MET2003,CNTRY,GQ,GQTYPE,GQTYPED,FARM,OWNERSHP,OWNERSHPD,MORTGAGE,MORTGAG2,COMMUSE,FARMPROD,ACREHOUS,MORTAMT1,MORTAMT2,TAXINCL,INSINCL,PROPINSR,PROPTX99,OWNCOST,RENT,RENTGRS,RENTMEAL,CONDOFEE,MOBLHOME,MOBLHOM2,MOBLOAN,SECRES,SECRESMO,SECRESRE,COSTELEC,COSTGAS,COSTWATR,COSTFUEL,PUBHOUS,RENTSUB,HEATSUB,LUNCHSUB,FOODSTMP,FDSTPAMT,VALUEH,LINGISOL,VACANCY,KITCHEN,KITCHENORIG,FRIDGE,FRIDGEORIG,SINK,STOVE,ROOMS,ROOMSORIG,PLUMBING,HOTWATER,SHOWER,TOILET,BUILTYR,BUILTYR2,UNITSSTR,BEDROOMS,BEDROOMSORIG,PHONE,PHONEORIG,CILAPTOP,CIHAND,CIOTHCOMP,CINETHH,CIMODEM,CISAT,CIDSL,CIFIBER,CIBRDBND,CIDIAL,CIOTHSVC,FUELHEAT,VEHICLES,SSMC,NFAMS,NSUBFAM,NCOUPLES,NMOTHERS,NFATHERS,MULTGEN,MULTGEND,CBNSUBFAM,REPWT1,REPWT2,REPWT3,R

In [12]:
results

TCuDataFrame(df_handle=b'\x80\xf7\xa6\x04\x00\x00\x00\x00"\x00\x00\x00\x00\x00\x00\x00`\xf8\xf1\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\xb0\x00\xd0\xc1\xa6\x00\x00\\', df_size=15857760, schema=b'\x10\x00\x00\x00\x0c\x00\x0e\x00\x06\x00\x05\x00\x08\x00\x00\x00\x0c\x00\x00\x00\x00\x01\x01\x00\x10\x00\x00\x00\x00\x00\n\x00\x08\x00\x00\x00\x04\x00\x00\x00\n\x00\x00\x00\x04\x00\x00\x00\xbf\x01\x00\x00\xac\x91\x00\x00D\x91\x00\x00\xec\x90\x00\x00\x98\x90\x00\x00D\x90\x00\x00\xf0\x8f\x00\x00\x9c\x8f\x00\x00H\x8f\x00\x00\xf4\x8e\x00\x00\x9c\x8e\x00\x00L\x8e\x00\x00\xfc\x8d\x00\x00\xa8\x8d\x00\x00P\x8d\x00\x00\xf8\x8c\x00\x00\xa8\x8c\x00\x00T\x8c\x00\x00\x04\x8c\x00\x00\xb4\x8b\x00\x00`\x8b\x00\x00\x10\x8b\x00\x00\xbc\x8a\x00\x00l\x8a\x00\x00\x1c\x8a\x00\x00\xcc\x89\x00\x00|\x89\x00\x00(\x89\x00\x00\xd4\x88\x00\x00\x80\x88\x00\x00,\x88\x00\x00\xd8\x87\x00\x00\x88\x87\x00\x008\x87\x00\x00\xe4

Use Numba to access the IPC memory handle

Note: this requires numba 0.32.0 + PR #2023

```bash
git clone https://github.com/numba/numba
cd numba
git fetch origin pull/2023/merge:pr/2023
git checkout pr/2023

python setup.py install
```

In [13]:
from numba import cuda
from numba.cuda.cudadrv import drvapi

In [14]:
ipc_handle = drvapi.cu_ipc_mem_handle(*results.df_handle)
ipch = cuda.driver.IpcHandle(None, ipc_handle, size=results.df_size)
ctx = cuda.current_context()
dptr = ipch.open(ctx)

dptr

`dptr` is GPU memory containing the query result

Convert `dptr` into a GPU device ndarray (numpy array like object on GPU)

In [19]:
import numpy as np

In [20]:
dtype = np.dtype(np.byte)
darr = cuda.devicearray.DeviceNDArray(shape=dptr.size, strides=dtype.itemsize, dtype=dtype, gpu_data=dptr)

Use PyGDF to read the arrow metadata from the query

In [22]:
from pygdf.gpuarrow import GpuArrowReader

In [23]:
reader = GpuArrowReader(darr)

Wrap result in a Python CUDA DataFrame

In [24]:
from pygdf.dataframe import DataFrame

In [25]:
df = DataFrame(reader.to_dict().items())

In [26]:
df.columns

('INCEARN',
 'RECTYPE',
 'IPUMS_YEAR',
 'DATANUM',
 'SERIAL',
 'NUMPREC',
 'SUBSAMP',
 'HHWT',
 'HHTYPE',
 'REPWT',
 'ADJUST',
 'CPI99',
 'REGION',
 'STATEICP',
 'STATEFIP',
 'COUNTY',
 'COUNTYFIPS',
 'METRO',
 'METAREA',
 'METAREAD',
 'MET2013',
 'MET2013ERR',
 'CITY',
 'CITYERR',
 'CITYPOP',
 'PUMA',
 'PUMARES2MIG',
 'STRATA',
 'PUMASUPR',
 'CONSPUMA',
 'CPUMA0010',
 'APPAL',
 'APPALD',
 'HOMELAND',
 'MET2003',
 'CNTRY',
 'GQ',
 'GQTYPE',
 'GQTYPED',
 'FARM',
 'OWNERSHP',
 'OWNERSHPD',
 'MORTGAGE',
 'MORTGAG2',
 'COMMUSE',
 'FARMPROD',
 'ACREHOUS',
 'MORTAMT1',
 'MORTAMT2',
 'TAXINCL',
 'INSINCL',
 'PROPINSR',
 'PROPTX99',
 'OWNCOST',
 'RENT',
 'RENTGRS',
 'RENTMEAL',
 'CONDOFEE',
 'MOBLHOME',
 'MOBLHOM2',
 'MOBLOAN',
 'SECRES',
 'SECRESMO',
 'SECRESRE',
 'COSTELEC',
 'COSTGAS',
 'COSTWATR',
 'COSTFUEL',
 'PUBHOUS',
 'RENTSUB',
 'HEATSUB',
 'LUNCHSUB',
 'FOODSTMP',
 'FDSTPAMT',
 'VALUEH',
 'LINGISOL',
 'VACANCY',
 'KITCHEN',
 'KITCHENORIG',
 'FRIDGE',
 'FRIDGEORIG',
 'SINK',
 'STOVE'

Preprocess the data

In [27]:
num_cols = set()
cat_cols = set()
response_set = set(['INCEARN'])
feature_names = set(df.columns) - response_set

Determine categorical and numeric columns.  Compute unique values from categorical columns.

In [28]:
uniques = {}
for k in feature_names:
    try:
        uniquevals = df[k].unique_k(k=1000)
        uniques[k] = uniquevals
    except ValueError:
        # more than 1000 unique values
        num_cols.add(k)
    else:
        # within 1000 unique values
        nunique = len(uniquevals)
        if nunique < 2:
            del df[k]        # drop constant column
        elif 1 < nunique < 1000:
            cat_cols.add(k)  # as cat column
        else:
            num_cols.add(k)  # as num column

Fix numeric columns.  Fill NA, Scale to [0, 1]. Drop near constant

In [29]:
for k in (num_cols - response_set):
    df[k] = df[k].fillna(df[k].mean())
    assert df[k].null_count == 0
    std = df[k].std()
    # drop near constant columns
    if not np.isfinite(std) or std < 1e-4:
        del df[k]
        print('drop near constant', k)
    else:
        df[k] = df[k].scale()

Expand categorical columns

In [30]:
for k in cat_cols:
    cats = uniques[k][1:]  # drop first
    df = df.one_hot_encoding(k, prefix=k, cats=cats)
    del df[k]

Ensure INCEARN is float64

In [31]:
df['INCEARN'] = df['INCEARN'].astype(np.float64)

In [32]:
# Print dtypes
{df[k].dtype for k in df.columns}

{dtype('float64')}

Create 60-40: training - testing

In [33]:
n60 = int(len(df) * 0.6)
print('60% of {} is {}'.format(len(df), n60))
train_df = df.loc[:n60]
test_df = df.loc[n60:]
print('train_df has {} rows | test_df has {} rows'.format(len(train_df), len(test_df)))

60% of 5568 is 3340
train_df has 3340 rows | test_df has 2228 rows


Turn the dataframes into a matrix

In [34]:
df.columns[0]

'INCEARN'

In [35]:
train_data_mat = train_df.as_gpu_matrix(columns=df.columns[1:])
train_result_mat = train_df.as_gpu_matrix(columns=[df.columns[0]])
test_data_mat = test_df.as_gpu_matrix(columns=df.columns[1:])
test_result_mat = test_df.as_gpu_matrix(columns=[df.columns[0]])

In [36]:
train_data_mat.shape, train_result_mat.shape, test_data_mat.shape, test_result_mat.shape

((3340, 8549), (3340, 1), (2228, 8549), (2228, 1))

Cleanup the IPC handle.

Note: `.as_gpu_matrix()` has created new copies; thus, we can close the IPC handle.

In [37]:
ipch.close()

The ctypes pointer to the gpu matrices

In [38]:
train_data_mat_ptr = train_data_mat.device_ctypes_pointer
train_result_mat_ptr = train_result_mat.device_ctypes_pointer
test_data_mat_ptr = test_data_mat.device_ctypes_pointer
test_result_mat_ptr = test_result_mat.device_ctypes_pointer

In [39]:
print('train_data_mat_ptr address', hex(train_data_mat_ptr.value))
print('train_result_mat_ptr address', hex(train_result_mat_ptr.value))
print('test_data_mat_ptr address', hex(test_data_mat_ptr.value))
print('test_result_mat_ptr address', hex(test_result_mat_ptr.value))

train_data_mat_ptr address 0x10223000000
train_result_mat_ptr address 0x10222ff4400
test_data_mat_ptr address 0x10230a00000
test_result_mat_ptr address 0x10222ffae00
